### CS102/CS103

Prof. Götz Pfeiffer<br />
School of Mathematics, Statistics and Applied Mathematics<br />
NUI Galway

# Lecture 20: Public Key Encryption

Modular arithmetic is prominently applied in encryption.
A popular scheme is the [RSA cryptosystem](https://en.wikipedia.org/wiki/RSA_%28cryptosystem%29), which consists of a pair
of keys, a **public key** to encrypt messages, and a **private key** to decrypt a received secret message. The security of this system
relies on the difficulty of factorising large numbers.

Here, we define a `Key` class that can provide both private and
public keys.  

##  RSA Encryption

 A **key pair** can be generated as follows:
 
 * Pick two (large) **prime numbers**, $p$ and $q$ say
 (and keep them secret).
 
 * Calculate their **product** $m = p q$ (which will be
 made public).
 
 * Compute $r = \phi(m) = (p-1)(q-1)$ (and keep it secret).
 (Here, $\phi(m)$ is [Euler's totient function](https://en.wikipedia.org/wiki/Euler%27s_totient_function).)
 
 * Pick a number $e$ with $\gcd(r, e) = 1$ (so $e$ has an inverse
 modulo $r$).
 
 * Compute the **inverse** $d$ of $e$ (modulo $r$).
 
 The **public key** then is the pair $(e, m)$ and the **private
 key** is the pair $(d, m)$ (with $d$ kept secret).
 
 * **Encryption** is a function 
 $$E \colon x \mapsto x^e \pmod{m}.$$
 
 * **Decryption** is a function 
 $$D \colon x \mapsto x^d \pmod{m}.$$
 
[Euler's Theorem](https://en.wikipedia.org/wiki/Euler%27s_theorem) states that, if $\gcd(n, a) = 1$ then
$$
a^{\phi(n)} \equiv 1 \pmod{n}.
$$
 
Hence, $D(E(x)) = x^{ed} = x^{1 + kr} = x \cdot (x^r)^k \equiv x \pmod{m}$, as $r = \phi(m)$. 

##  Fast Exponentiation

In order to apply these encryption and decryption methods, it helps to have an **efficient exponentiation** algorithm for possibly large
exponents, modulo $m$.

The [binary method](https://en.wikipedia.org/wiki/Modular_exponentiation) for modular exponentiation is an adaption of a method known as [Russian peasant multiplication](https://en.wikipedia.org/wiki/Ancient_Egyptian_multiplication#Russian_peasant_multiplication).

We can implement this as a **special method** `__pow__()` of a class `Residue` (defined in a file `residue.py`), whose instances
represent residue classes modulo $m$, with instance variables `val` 
and `mod`.

```python
def __pow__(self, exp):
    "self ** exp"
    if exp == 0:
        return Residue(1, self.mod)
    elif exp == 1:
        return self
    else:
        q, r = divmod(exp, 2) # exp == 2 * q + r, r < 2
        result = (self * self) ** q
        if r > 0:
            result *= self
        return result
```

In [1]:
from residue import *
a = Residue(17, 1234689)

In [2]:
a**78678

Residue(400912, 1234689)

## Keys

A **key**, like a residue class, is an object that has
a modulus and a value as attributes. We call the value
the **exponent** of the key, and store modulus and exponent
in **instance variables** `mod` and `exp`.  These are the data
a key instance **knows**.

Then there is one thing a key can **do**: it can transform
a given code (a number) into a new one, by raising it into the power
of its exponent, modulo its modulus.  We implement this as
a **method** `transform` of the following `Key` class.

In [3]:
class Key:
    "RSA encryption and decryption"

    def __init__(self, residue):
        "constructor"
        self.mod = residue.mod
        self.exp = residue.val

    def transform(self, code):
        "encrypt or decrypt"
        return (Residue(code, self.mod) ** self.exp).val


For example, ...

In [4]:
p, q = 17, 23
m, r, e = p * q, (p - 1) * (q - 1), 123
res = Residue(e, r)

In [5]:
res

Residue(123, 352)

In [6]:
d = res.inverse().val
d

83

In [7]:
public = Key(Residue(e, m)) 
private = Key(Residue(d, m))

In [8]:
message = 283
secret = public.transform(message)
secret

250

In [9]:
private.transform(secret)

283

## Generating Key Pairs

In [10]:
def primes_upto(limit):
    "generator: all primes up to limit"
    is_prime = [False, False] + [True] * (limit - 2)
    for n in range(2, limit):
        if is_prime[n]:
            yield n
            for an in range(n*n, limit, n):
                is_prime[an] = False

In [11]:
def primes_range(start, stop):
    "list all primes in range(start, stop)"
    if start < stop:
        primes = primes_upto(stop)
        for p in primes:
            if p >= start:
                return [p] + list(primes)
    return []

In [12]:
size = 10**6
primes = primes_range(size, 2*size)
len(primes)

70435

In [13]:
import random
p, q = random.choice(primes), random.choice(primes)
p, q

(1277449, 1050053)

In [14]:
m, r = p * q, (p - 1) * (q - 1)
m, r

(1341389154797, 1341386827296)

In [15]:
e = random.randrange(r)

In [16]:
e, gcd(r, e)

(555445400130, 6)

In [17]:
while gcd(r, e) > 1:
    e = random.randrange(r)
    print(gcd(r, e))

68
1


In [18]:
e

313875753791

In [19]:
d = Residue(e, r).inverse().val
d

721141325375

In [20]:
public = Key(Residue(e, m)) 
private = Key(Residue(d, m))

In [21]:
message = random.randrange(m)
message

652427520030

In [22]:
secret = public.transform(message)
secret

611681437758

In [23]:
private.transform(secret)

652427520030

## Summary: Encryption

* Modular **exponentiation** can be done in a way that keeps intermediate results small.

* Encryption and decryption methods can be implemented as
**methods** of suitable `python` classes.

* Function/method **bodies** need not be long.

* In a **public key encryption** system, the encryption key
can be made public, without compromising the security of
the system, as long as the decryption key is kept private.

* **Factorization** (in contrast to multiplication) is **computationally hard**.

* The **RSA cryptosystem** relies on the difficulty of
recovering $p$ and $q$ from the product $p q$.

* The Euler **totient function** $\phi(m)$ can only
be computed if the **prime factorization** of $m$ is known.

* **Euler's theorem** implies that if $e$ and $d$ are
inverse to each other modulo $\phi(m)$ then
$(x^e)^d = x^{ed} \equiv x \pmod{m}$, i.e., exponentiation with
$e$ and $d$ undo each other.